In [ ]:
抓了70+数据后就突然不work了

In [2]:
from urllib.request import Request, urlopen
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import numpy as np
import time
import random
import os
import json

In [40]:
class MyWalmart():
    def get_soup(self, url):
        """
        This function get the beautifulsoup object of a webpage.

        Args:
            url (str): the link string of webpage

        Returns:
            soup (obj): beautifulsoup object
        """
        request = Request(url, headers={'User-Agent': 'Resistance is futile'})
        response = urlopen(request)
        return BeautifulSoup(response, "html.parser")
    
    def get_product_links_from_page(self, url):
        """
        This function gets the links of the product on the product search page page.

        Args:
            url (str): link to joblist page

        Returns:
            links (list): list of links to individual products of the keyword
        """
             
        soup = self.get_soup(url)
        head = "http://www.walmart.com"  
        links = []
        for item in soup.find_all("a", href=True):
            if 'link-identifier=' in str(item) and '/ip/' in str(item):  
                link = item['href']
                links.append(head+link)
        return links
     
    def get_products(self, keyword):
        """
        Args:
            keyword (str): example: 'Christmas tree'

        Returns:
        """
        gap_min = 5 #min sleep time between clicks
        gap_max = 10 #max sleep time between clicks
        
        url = "https://www.walmart.com/search?q="
        keyword_list = keyword.split()
        
        for i in range(len(keyword_list)):
            url+='%20'
            url+=keyword_list[i]    

            #get links individual products with the keyword
        links = self.get_product_links_from_page(url)
        print(links)
        #get information of individual products
        for link in links:
            gap = np.random.uniform(gap_min,gap_max) 
            time.sleep(gap)
            data = self.get_info_from_product_page(link)

            print(json.dumps(data))
            
    def get_info_from_product_page(self, url):
        """
        This function get info from the webpage of an individual product.

        Args:
            url (str): link to the product webpage

        Returns:
            data (dict): dictionary with keywords: 
                         item, price, link
        """
        soup = self.get_soup(url)
        data = {}
        
        try:
            data['item'] = soup.find('h1', itemprop="name").getText()
            data["price"] = soup.find('span', itemprop="price").getText()
            data["link"] = str(url)
        except:
            pass
        return data
